# Client Recommendation

* Frequent Pattern mining doesn't work well for this data set.
* Therefore I'm using similarity recommendation with collaborative filtering.
  * The assumption here is, a client will purchase similar products they had purchased before.

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
all_order_train = pd.read_pickle('../all_order_train.pkl')
all_order_test = pd.read_pickle('../all_order_test.pkl')
print(all_order_train.shape, all_order_test.shape)

all_order_train.head()

(33720820, 12) (98286, 12)


,order_id,user_id,order_number,order_hour_of_day,product_id,purchase_date,merchant,product_name,price,aisle,department,week_number
0,2539329,1,1,8,196,2019-02-13,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,7
1,2231262,31,17,11,196,2019-03-10,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
2,3058369,195,34,10,196,2019-02-25,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,9
3,2257155,951,3,20,196,2019-03-09,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
4,1121647,992,7,10,196,2019-03-08,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10


In [8]:
shared_clients = set(all_order_train['user_id'].values).intersection(all_order_test['user_id'].values)
print(len(shared_clients))

3457


In [10]:
train_shared_client_week_df = all_order_train.loc[all_order_train['user_id']\
                                                  .isin(list(shared_clients))][['user_id', 'week_number']]\
                                                  .astype('str').drop_duplicates()\
                                                  .groupby(['user_id'], as_index=False)['week_number']\
                                                  .agg(['count']).reset_index()\
                                                  .sort_values(['count'], ascending=False)

train_shared_client_week_df.head()

,user_id,count
1549,183503,5
3210,86398,5
290,116891,5
3196,85846,5
1494,180563,5


In [11]:
target_client = 183503

## Client Match Score & Discount

In [27]:
merchant_sales_df = all_order_train[['merchant', 'user_id', 'price']].drop_duplicates()\
          .groupby(['merchant'], as_index=False)\
          .agg({'user_id': 'count', 'price': 'sum'})

merchant_sales_df.head()

,merchant,user_id,price
0,49th Parallel Grocery,1211766,1.452954e+07
1,Asian Food Centre,1210187,1.451086e+07
2,Askew's Foods,1203583,1.443146e+07
3,Avril (Health Supermarket),1200036,1.439485e+07
4,Calgary Co-op,1017673,1.218554e+07


In [28]:
merchant_sales_df['avg_sales_per_client'] = round(merchant_sales_df['price']/merchant_sales_df['user_id'], 4)

merchant_sales_df.head()

,merchant,user_id,price,avg_sales_per_client
0,49th Parallel Grocery,1211766,1.452954e+07,11.9904
1,Asian Food Centre,1210187,1.451086e+07,11.9906
2,Askew's Foods,1203583,1.443146e+07,11.9904
3,Avril (Health Supermarket),1200036,1.439485e+07,11.9953
4,Calgary Co-op,1017673,1.218554e+07,11.9739


In [29]:
client_purchase_df = all_order_train[['user_id', 'price', 'order_id']].drop_duplicates()\
          .groupby(['user_id'], as_index=False)\
          .agg({'price': 'sum', 'order_id': 'count'})

client_purchase_df.head()

,user_id,price,order_id
0,1,560.43,70
1,2,2685.71,226
2,3,994.48,88
3,4,282.01,18
4,5,543.37,46


In [30]:
client_purchase_df['avg_purchase'] = round(client_purchase_df['price']/client_purchase_df['order_id'], 4)

client_purchase_df.head()

,user_id,price,order_id,avg_purchase
0,1,560.43,70,8.0061
1,2,2685.71,226,11.8837
2,3,994.48,88,11.3009
3,4,282.01,18,15.6672
4,5,543.37,46,11.8124


In [34]:
merchant_sales_df['merchant_match_score'] = abs(1 - 
                            client_purchase_df.loc[client_purchase_df['user_id']==target_client]['avg_purchase'].values[0]\
                                                /merchant_sales_df['avg_sales_per_client'])
merchant_sales_df['discount'] = merchant_sales_df['price'] * 0.0000007

merchant_sales_df = merchant_sales_df.sort_values(by='merchant_match_score')
merchant_sales_df.head()

,merchant,user_id,price,avg_sales_per_client,merchant_match_score,discount
44,Sunterra Market,87915,1042779.22,11.8612,0.158803,0.729945
45,TaiKo Supermarket,79776,948310.74,11.8872,0.160643,0.663818
37,Planet Organic,155315,1848801.80,11.9036,0.161800,1.294161
36,Panchvati Supermarket,189487,2257196.69,11.9121,0.162398,1.580038
25,Lococo's,352710,4205428.28,11.9232,0.163178,2.943800


In [32]:
merchant_lst = merchant_sales_df['merchant'].values[0:10]
merchant_lst

array(['Sunterra Market', 'TaiKo Supermarket', 'Planet Organic',
       'Panchvati Supermarket', "Lococo's", "Nature's Emporium",
       "Mike Dean's Super Food Stores", 'Marché Frais', "Vince's Market",
       'Pomme Natural Market'], dtype=object)

## Collaborative Filtering